In [ ]:
from astropy.coordinates import SkyCoord
from hips import WCSGeometry, make_sky_image
from hips import HipsSurveyProperties
from ipywidgets import Layout, Box, widgets


# HIPS Image

In [ ]:
url = 'http://surveys.roe.ac.uk/hips71/LAS/LAS_Y_OUT//properties'
hips_survey = 'CDS/P/DSS2/red'

In [ ]:
print (url)

In [ ]:
hips_survey_property = HipsSurveyProperties.fetch(url)

In [ ]:
geometry = WCSGeometry.create(skydir=SkyCoord(217.9, -2.3, unit='deg', frame='fk5'),width=180, height=180, fov='0.02 deg',coordsys='icrs', projection='TAN')
from hips import make_sky_image
result = make_sky_image(geometry, hips_survey_property, 'fits') 

In [ ]:
result.plot()

In [ ]:
#result.image

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
from astropy.visualization.mpl_normalize import simple_norm

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(1, 1, 1)
norm = simple_norm(result.image, 'sqrt', min_percent=1, max_percent=99)
im = ax.imshow(result.image, origin='lower', norm=norm, cmap='gray_r')
fig.colorbar(im)

## Firethorn Query

In [ ]:
import firethorn
import time

In [ ]:
ft = firethorn.Firethorn(endpoint='http://tap.roe.ac.uk/firethorn')

In [ ]:
for i in ft.get_public_resource_names():
    print (i)

In [ ]:
resource = ft.get_public_resource_by_name('WSA')

In [ ]:
schema = resource.get_schema_by_name("UKIDSSDR9PLUS")
print (schema.get_table_names())

In [ ]:
my_workspace = ft.new_workspace("mySpace")
resource.get_schema_by_name(
        "UKIDSSDR9PLUS"
    )

In [ ]:
# Import a Schema from the OSA workspace
my_workspace.add_schema(
    resource.get_schema_by_name(
        "UKIDSSDR9PLUS"
    )
)


In [ ]:
# Run a Synchronous query
query_text = """
SELECT TOP 500 
sourceID, ra,dec as "decl"  FROM 
    UKIDSSDR9PLUS.lasSource
WHERE 
    ra >210 AND ra<220 AND dec <-2.0 AND dec > -3.0    
ORDER by dec
"""
query = my_workspace.query(query_text)

In [ ]:
# Get results table
table = query.results()


In [ ]:
# Get results table row count
table.rowcount()

In [ ]:
# Get results table as astropy table
astropy_table = table.as_astropy()
table.as_astropy()

## Scatter Plot using Plotly

In [ ]:
from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.graph_objs import *
import plotly.graph_objs as go

import numpy as np

init_notebook_mode() # Important

In [ ]:
#Large Datasets

N = 100000
trace = go.Scattergl(
    x = astropy_table["ra"],
    y = astropy_table["decl"],
    mode = 'markers',
    marker = dict(
        color = '#FFBAD2',
        line = dict(width = 1)
    )
)
data = [trace]
iplot(data, filename='compare_webgl')

## Plot points in Sky

In [ ]:
import ipyaladin as ipyal

In [ ]:
aladin=ipyal.Aladin(target='14 20 0.609 -02 33 49.24', fov=14, survey='http://surveys.roe.ac.uk/hips71/LAS/LAS_Y_OUT')
aladin

In [ ]:

aladin.add_table(astropy_table)


from IPython.display import display, HTML
js = "<script> aladin.setImageSurvey(aladin.createImageSurvey('BIG_color', 'Color BiH HiPS', 'http://surveys.roe.ac.uk/hips71/LAS/LAS_Y_OUT//')); // setting a custom HiPS</script>"
display(HTML(js))

dir(aladin)

## Linked Catalogue Views


In [ ]:
from ipywidgets import Layout, Box, widgets

a = ipyal.Aladin(layout=Layout(width='33.33%'), target='14 20 0.609 -02 33 49.24', fov=0.3, survey='http://surveys.roe.ac.uk/hips71/LAS/LAS_Y_OUT//properties')
b = ipyal.Aladin(layout=Layout(width='33.33%'), survey='P/DSS2/red')
c = ipyal.Aladin(layout=Layout(width='33.33%'), survey='P/2MASS/color')

# synchronize target between 3 widgets
widgets.jslink((a, 'target'), (b, 'target'))
widgets.jslink((b, 'target'), (c, 'target'))

# synchronize FoV (zoom level) between 3 widgets
widgets.jslink((a, 'fov'), (b, 'fov'))
widgets.jslink((b, 'fov'), (c, 'fov'))

items = [a, b, c]

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    border='solid',
                    width='100%')
box = Box(children=items, layout=box_layout)
box

## Getting Light Curves and Object name


In [ ]:
from ipyaladin import Aladin

from ipywidgets import Layout, Box, widgets

aladin = Aladin(layout=Layout(width='70%'), target='14 20 0.609 -02 33 49.24', fov=0.3)
info = widgets.HTML()


box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%')
box = Box(children=[aladin, info], layout=box_layout)
box

In [ ]:

import requests
def process_result(data):
    info.value = ''
    ra = data['ra']
    dec = data['dec']
    radius = min(aladin.fov / 10, 5)
    query = """SELECT TOP 1 main_id, ra, dec, DISTANCE(POINT('ICRS', %f, %f), POINT('ICRS', ra, dec)) as d FROM basic
               WHERE CONTAINS(POINT('ICRS', ra, dec), CIRCLE('ICRS', %f, %f, %f))=1
               ORDER BY d ASC""" % (ra, dec, ra, dec, aladin.fov / 10)
    
    r = requests.get('http://simbad.u-strasbg.fr/simbad/sim-tap/sync', params={'query': query, 'request': 'doQuery', 'lang': 'adql', 'format': 'json', 'phase': 'run'})
    obj_name = ''
    obj_coo = None
    obj_data = r.json()['data']
    if len(obj_data)==0:
        return
    
    obj_name = obj_data[0][0]
    obj_coo = [obj_data[0][1], obj_data[0][2]]
        
    sed_img = '<img src="http://cdsportal.u-strasbg.fr/cgi-bin/PhotVizPreview/plot?ra=%f&dec=%f&radius_arcsec=5&w=200&h=150&point_size=4">' % (obj_coo[0], obj_coo[1])
    info.value =  '<h2>%s</h2><br><br>%s' % (obj_name, sed_img)
    
aladin.add_listener('click', process_result)

## Selecting points in sky


In [ ]:

from ipyaladin import Aladin
from ipywidgets import Layout, Box, widgets

aladin = Aladin(layout=Layout(width='50%'), target='14 20 6.807 -02 32 26.23', fov=0.3,survey='http://surveys.roe.ac.uk/hips71/LAS/LAS_Y_OUT//properties')


button = widgets.Button(description="Select")
def on_button_clicked(b):
    aladin.rectangular_selection()

button.on_click(on_button_clicked)
table_info = widgets.HTML(layout=Layout(height='auto', overflow='auto'))


box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%',
                    position='relative',
                    overflow='hidden',
                    height='100vh',
                    margin='-100px 0 0 0',
                    padding='100px 0 0 0 '
                   )
box = Box(children=[aladin, button, table_info], layout=box_layout)
box

In [ ]:
from astroquery.simbad import Simbad
import astropy.units as u

aladin.add_table(astropy_table)

def process_result(sources):
    s = '<table border="1">'
    s += '<tr><th>Source ID</th><th>RA</th><th>DEC</th></tr>'
    for source in sources:
        s += '<tr><td>%s</td><td>%s</td><td>%s</td></tr>' % (source['data']['sourceID'], source['data']['ra'], source['data']['decl'])
    s += '</table>'
    table_info.value = s
    
aladin.add_listener('select', process_result)

# Select points on the sky, run query on TAP service

In [ ]:

from ipyaladin import Aladin
from ipywidgets import Layout, Box, widgets

aladin = Aladin(layout=Layout(width='50%'), target='14 20 6.807 -02 32 26.23', fov=0.3,survey='http://surveys.roe.ac.uk/hips71/LAS/LAS_Y_OUT//properties')


button = widgets.Button(description="Select")
def on_button_clicked(b):
    aladin.rectangular_selection()

button.on_click(on_button_clicked)
table_info = widgets.HTML(layout=Layout(height='auto', overflow='auto'))


input_text = widgets.IntText(
    value=7,
    description='Any:',
    disabled=False
)

box_layout = Layout(display='flex',
                    flex_flow='row',
                    align_items='stretch',
                    width='100%',
                    position='relative',
                    overflow='hidden',
                    height='100vh',
                    margin='-100px 0 0 0',
                    padding='100px 0 0 0 '
                   )
box = Box(children=[aladin, button, table_info], layout=box_layout)
box

In [ ]:
from astroquery.simbad import Simbad
import astropy.units as u
from pyvo.dal import tap
from astroquery.utils.tap.core import TapPlus

aladin.add_table(astropy_table)


def process_result(sources):
    query_text = "SELECT sourceID, ra, dec FROM lasSource WHERE "

    for source in sources:
        query_text+="(ra=" + str(source['data']['ra'])
        query_text+=" AND dec=" + str(source['data']['decl']) + ")"
        query_text+=" OR "


    query_text = query_text[:-3]
    service = TapPlus(url="http://wfaudata.roe.ac.uk/ukidssDR9-dsa/TAP")
    job = service.launch_job(query_text)
    astropy_table = job.get_results()
    
    s = "<h1>TAP Query results:</h1><br/> http://wfaudata.roe.ac.uk/ukidssDR5-dsa/TAP"
    s += '<table border="1">'
    s += '<tr><th>Source ID</th><th>RA</th><th>DEC</th></tr>'
    
    table_info.value = str(len(astropy_table))
    table_info.value = query_text

    for row in astropy_table:
        s += '<tr><td>%s</td><td>%s</td><td>%s</td></tr>' % (row['sourceID'], row['ra'], row['dec'])
    s += '</table>' 
    table_info.value = s

    #aladin.add_table(astropy_table)

    
    
aladin.add_listener('select', process_result)